In [1]:
import librosa
import soundfile
import os, glob, pickle
import pyttsx3
import numpy as np
import pyaudio
import numpy
import speech_recognition as sr
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import os

In [2]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        '''if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))'''
    return result

In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
import joblib
file = '../SRE_Nurel_Network/savedModels/model.joblib'
load_model= joblib.load(file)

In [4]:
speaker = pyttsx3.init()
speaker.setProperty("rate", 180)


listener = sr.Recognizer()
def talk(text):
    speaker.say(text)
    speaker.runAndWait()

def get_info() :      #function for geeting voice from user and then  converting it into the text
    with sr.Microphone() as source:
        print('listening...')
        voice = listener.record(source,duration=4)

        try:
            info = listener.recognize_google(voice)
            #print(format(info))
            return info.lower()

        except:
            talk('did not hear properly please speak loudly')
            get_info()

talk("if you want to test the model on file from system speak system else speak live for recording your audio");

option=get_info()
print(option)

talk("start speaking something I am ready to record your audio")
RATE=16000
RECORD_SECONDS = 15
CHUNKSIZE = 1024
# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)
frames = [] # A python-list of chunks(numpy.ndarray)
for _ in range(0, int(RATE / CHUNKSIZE * RECORD_SECONDS)):
    data = stream.read(CHUNKSIZE)
    frames.append(numpy.fromstring(data, dtype=numpy.int16))
#Convert the list of numpy-arrays into a 1D array (column-wise)
numpydata = numpy.hstack(frames)
# close stream
stream.stop_stream()
stream.close()
p.terminate()
import scipy.io.wavfile as wav
wav.write('out.wav',RATE,numpydata)
talk("your audio is recorded wait for some time to get results")
file="out.wav"

listening...
hello


C:\Users\ramra\AppData\Local\Temp\ipykernel_12216\507792526.py:39: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  frames.append(numpy.fromstring(data, dtype=numpy.int16))


In [5]:
file = 'out.wav'

In [35]:
fe = extract_feature(file,True,True,True)

In [42]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("F:\\PROJECTS\\SRE_Nurel_Network\\RAVDESS_Emotional_speech_audio\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=20)

In [43]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [44]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [45]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 52


In [46]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(900,), learning_rate='adaptive', max_iter=500)

In [55]:
#DataFlair - Train the model
model.fit(x_train,y_train)
'''x = x_train[:None]
model.fit(x,y_train)'''

'x = x_train[:None]\nmodel.fit(x,y_train)'

In [62]:
#DataFlair - Predict for the test set
y_pred=load_model.predict([fe])

In [63]:
y_pred

array(['happy'], dtype='<U9')

In [ ]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

from joblib import dump

joblib.dump(model, 'model.joblib')